In [1]:
import pandas as pd
import sqlite3

# Load datasets
menu_items_df = pd.read_csv('/content/menu_items.csv')
order_details_df = pd.read_csv('/content/order_details.csv')

# Create a SQLite database
conn = sqlite3.connect('restaurant_orders.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS menu_items (
    menu_item_id INTEGER PRIMARY KEY,
    item_name TEXT,
    category TEXT,
    price REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS order_details (
    order_details_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    order_date TEXT,
    order_time TEXT,
    item_id INTEGER,
    FOREIGN KEY(item_id) REFERENCES menu_items(menu_item_id)
)
''')

# Insert data into tables
menu_items_df.to_sql('menu_items', conn, if_exists='append', index=False)
order_details_df.to_sql('order_details', conn, if_exists='append', index=False)

conn.commit()



In [2]:
# Retrieve all columns from the menu_items table
query1 = "SELECT * FROM menu_items LIMIT 5"
print(pd.read_sql_query(query1, conn))



   menu_item_id      item_name  category  price
0           101      Hamburger  American  12.95
1           102   Cheeseburger  American  13.95
2           103        Hot Dog  American   9.00
3           104  Veggie Burger  American  10.50
4           105   Mac & Cheese  American   7.00


In [3]:
# Display the first 5 rows from the order_details table
query2 = "SELECT * FROM order_details LIMIT 5"
print(pd.read_sql_query(query2, conn))

   order_details_id  order_id order_date   order_time  item_id
0                 1         1   1/1/2023  11:38:36 AM      109
1                 2         2   1/1/2023  11:57:40 AM      108
2                 3         2   1/1/2023  11:57:40 AM      124
3                 4         2   1/1/2023  11:57:40 AM      117
4                 5         2   1/1/2023  11:57:40 AM      129


In [6]:
# Select the item_name and price columns for items in the 'Main Course' category, sorted by price in descending order
query3 = """
SELECT item_name, price
FROM menu_items
WHERE category = 'American'
ORDER BY price DESC
"""
print(pd.read_sql_query(query3, conn))


       item_name  price
0   Cheeseburger  13.95
1      Hamburger  12.95
2  Veggie Burger  10.50
3        Hot Dog   9.00
4   Mac & Cheese   7.00
5   French Fries   7.00


In [7]:
# Calculate the average price of menu items
query4 = "SELECT AVG(price) AS avg_price FROM menu_items"
print(pd.read_sql_query(query4, conn))




   avg_price
0  13.285937


In [8]:
# Find the total number of orders placed
query5 = "SELECT COUNT(DISTINCT order_id) AS total_orders FROM order_details"
print(pd.read_sql_query(query5, conn))

   total_orders
0          5370


In [9]:
# Retrieve the item_name, order_date, and order_time for all items in the order_details table
query6 = """
SELECT oi.item_name, od.order_date, od.order_time
FROM order_details od
JOIN menu_items oi ON od.item_id = oi.menu_item_id
"""
print(pd.read_sql_query(query6, conn))


              item_name order_date   order_time
0      Korean Beef Bowl   1/1/2023  11:38:36 AM
1         Tofu Pad Thai   1/1/2023  11:57:40 AM
2             Spaghetti   1/1/2023  11:57:40 AM
3       Chicken Burrito   1/1/2023  11:57:40 AM
4      Mushroom Ravioli   1/1/2023  11:57:40 AM
...                 ...        ...          ...
12092  Korean Beef Bowl  3/31/2023  10:05:04 PM
12093  Mushroom Ravioli  3/31/2023  10:05:04 PM
12094       Steak Torta  3/31/2023  10:05:04 PM
12095     Chips & Salsa  3/31/2023  10:05:04 PM
12096     Chips & Salsa  3/31/2023  10:15:48 PM

[12097 rows x 3 columns]


In [10]:
query8 = """
SELECT substr(order_date, 1, 2) AS month, COUNT(*) AS orders_count
FROM order_details
GROUP BY month
"""
print(pd.read_sql_query(query8, conn))


  month  orders_count
0    1/          4156
1    2/          3892
2    3/          4186


In [11]:
# Show the categories with the average price greater than $15 and include the count of items in each category
query9 = """
SELECT category, AVG(price) AS avg_price, COUNT(*) AS item_count
FROM menu_items
GROUP BY category
HAVING avg_price > 15
"""
print(pd.read_sql_query(query9, conn))


  category  avg_price  item_count
0  Italian      16.75           9


In [12]:
# Display the item_name and price, and indicate if the item is priced above $20 with a new column named 'Expensive'
query10 = """
SELECT item_name, price,
       CASE
           WHEN price > 20 THEN 'Yes'
           ELSE 'No'
       END AS Expensive
FROM menu_items
"""
print(pd.read_sql_query(query10, conn))


                item_name  price Expensive
0               Hamburger  12.95        No
1            Cheeseburger  13.95        No
2                 Hot Dog   9.00        No
3           Veggie Burger  10.50        No
4            Mac & Cheese   7.00        No
5            French Fries   7.00        No
6          Orange Chicken  16.50        No
7           Tofu Pad Thai  14.50        No
8        Korean Beef Bowl  17.95        No
9              Pork Ramen  17.95        No
10        California Roll  11.95        No
11            Salmon Roll  14.95        No
12                Edamame   5.00        No
13            Potstickers   9.00        No
14          Chicken Tacos  11.95        No
15            Steak Tacos  13.95        No
16        Chicken Burrito  12.95        No
17          Steak Burrito  14.95        No
18          Chicken Torta  11.95        No
19            Steak Torta  13.95        No
20     Cheese Quesadillas  10.50        No
21          Chips & Salsa   7.00        No
22      Chi

In [13]:
# Update the price of the menu item with item_id = 101 to $25
cursor.execute("UPDATE menu_items SET price = 25 WHERE menu_item_id = 101")
conn.commit()


In [14]:
# Insert a new record into the menu_items table for a dessert item
cursor.execute("""
INSERT INTO menu_items (menu_item_id, item_name, category, price)
VALUES (999, 'Chocolate Cake', 'Dessert', 6.50)
""")
conn.commit()


In [15]:
# Delete all records from the order_details table where the order_id is less than 100
cursor.execute("DELETE FROM order_details WHERE order_id < 100")
conn.commit()


In [16]:
# Rank menu items based on their prices, displaying the item_name and its rank
query13 = """
SELECT item_name, price,
       RANK() OVER (ORDER BY price DESC) AS price_rank
FROM menu_items
"""
print(pd.read_sql_query(query13, conn))


                item_name  price  price_rank
0               Hamburger  25.00           1
1           Shrimp Scampi  19.95           2
2        Korean Beef Bowl  17.95           3
3              Pork Ramen  17.95           3
4   Spaghetti & Meatballs  17.95           3
5            Meat Lasagna  17.95           3
6        Chicken Parmesan  17.95           3
7       Eggplant Parmesan  16.95           8
8          Orange Chicken  16.50           9
9          Cheese Lasagna  15.50          10
10       Mushroom Ravioli  15.50          10
11            Salmon Roll  14.95          12
12          Steak Burrito  14.95          12
13          Tofu Pad Thai  14.50          14
14              Spaghetti  14.50          14
15     Fettuccine Alfredo  14.50          14
16           Cheeseburger  13.95          17
17            Steak Tacos  13.95          17
18            Steak Torta  13.95          17
19        Chicken Burrito  12.95          20
20        California Roll  11.95          21
21        

In [17]:
# Display the item_name and the price difference from the previous and next menu item
query14 = """
SELECT item_name, price,
       price - LAG(price, 1) OVER (ORDER BY price) AS prev_price_diff,
       LEAD(price, 1) OVER (ORDER BY price) AS next_price_diff
FROM menu_items
"""
print(pd.read_sql_query(query14, conn))


                item_name  price  prev_price_diff  next_price_diff
0                 Edamame   5.00              NaN             6.50
1          Chocolate Cake   6.50             1.50             7.00
2            Mac & Cheese   7.00             0.50             7.00
3            French Fries   7.00             0.00             7.00
4           Chips & Salsa   7.00             0.00             9.00
5                 Hot Dog   9.00             2.00             9.00
6             Potstickers   9.00             0.00             9.00
7       Chips & Guacamole   9.00             0.00            10.50
8           Veggie Burger  10.50             1.50            10.50
9      Cheese Quesadillas  10.50             0.00            11.95
10        California Roll  11.95             1.45            11.95
11          Chicken Tacos  11.95             0.00            11.95
12          Chicken Torta  11.95             0.00            12.95
13        Chicken Burrito  12.95             1.00            1

In [18]:
# Create a CTE that lists menu items with prices above $15
query15_cte = """
WITH ExpensiveItems AS (
    SELECT item_name, price
    FROM menu_items
    WHERE price > 15
)
SELECT COUNT(*) AS count_expensive_items FROM ExpensiveItems
"""
print(pd.read_sql_query(query15_cte, conn))


   count_expensive_items
0                     11


In [ ]:
query15_cte = """
WITH ExpensiveItems AS (
    SELECT
        item_name
    FROM
        menu_items
    WHERE
        price > 15
)
SELECT
    COUNT(*) AS num_expensive_items
FROM
    ExpensiveItems;
"""
print(pd.read_sql_query(query15_cte, conn))


   num_expensive_items
0                   11


In [19]:
# Retrieve the order_id, item_name, and price for all orders with their respective menu item details, including rows even if there is no matching menu item
query16 = """
SELECT od.order_id, mi.item_name, mi.price
FROM order_details od
LEFT JOIN menu_items mi ON od.item_id = mi.menu_item_id
"""
print(pd.read_sql_query(query16, conn))


       order_id           item_name  price
0           100    Korean Beef Bowl  17.95
1           101        Cheeseburger  13.95
2           101             Edamame   5.00
3           101  Cheese Quesadillas  10.50
4           101       Chips & Salsa   7.00
...         ...                 ...    ...
11996      5369    Korean Beef Bowl  17.95
11997      5369    Mushroom Ravioli  15.50
11998      5369         Steak Torta  13.95
11999      5369       Chips & Salsa   7.00
12000      5370       Chips & Salsa   7.00

[12001 rows x 3 columns]


In [20]:
# Unpivot the menu_items table to show a list of menu item properties
query17 = """
SELECT menu_item_id, 'item_name' AS property, item_name AS value FROM menu_items
UNION
SELECT menu_item_id, 'category' AS property, category AS value FROM menu_items
UNION
SELECT menu_item_id, 'price' AS property, price AS value FROM menu_items
"""
print(pd.read_sql_query(query17, conn))


    menu_item_id   property              value
0            101   category           American
1            101  item_name          Hamburger
2            101      price               25.0
3            102   category           American
4            102  item_name       Cheeseburger
..           ...        ...                ...
94           132  item_name  Eggplant Parmesan
95           132      price              16.95
96           999   category            Dessert
97           999  item_name     Chocolate Cake
98           999      price                6.5

[99 rows x 3 columns]


In [21]:
import pandas as pd

# Write a dynamic SQL query that fetches original min and max prices within a category
def category_price_range(category):
    query = f"""
    SELECT MIN(price) as min_price, MAX(price) as max_price
    FROM menu_items
    WHERE category = '{category}'
    """
    return pd.read_sql_query(query, conn)

# Example usage
print(category_price_range('American'))


   min_price  max_price
0        7.0       25.0


In [22]:


# Write a dynamic SQL query that allows users to filter menu items based on category and price range
def dynamic_query(category, min_price, max_price):
    query = f"""
    SELECT item_name, price
    FROM menu_items
    WHERE category = '{category}' AND price BETWEEN {min_price} AND {max_price}
    """
    return pd.read_sql_query(query, conn)

# Example usage
print(dynamic_query('American', 10, 20))



       item_name  price
0   Cheeseburger  13.95
1  Veggie Burger  10.50


In [23]:
import pandas as pd

# Write a dynamic SQL query that allows users to filter menu items based on category and price range
def dynamic_query(category):
    subquery = f"""
    SELECT MIN(price) as min_price, MAX(price) as max_price
    FROM menu_items
    WHERE category = '{category}'
    """
    category_prices = pd.read_sql_query(subquery, conn)
    min_price = category_prices['min_price'].values[0]
    max_price = category_prices['max_price'].values[0]

    query = f"""
    SELECT item_name, price
    FROM menu_items
    WHERE category = '{category}' AND price BETWEEN {min_price} AND {max_price}
    """
    return pd.read_sql_query(query, conn)

# Example usage
print(dynamic_query('American'))


       item_name  price
0      Hamburger  25.00
1   Cheeseburger  13.95
2        Hot Dog   9.00
3  Veggie Burger  10.50
4   Mac & Cheese   7.00
5   French Fries   7.00


In [31]:
def get_average_price_by_category(category):
    query = f"""
    SELECT AVG(price) AS avg_price
    FROM menu_items
    WHERE category = '{category}'
    """
    return pd.read_sql_query(query, conn)

# Example usage
print(get_average_price_by_category('Asian'))


   avg_price
0     13.475
